## 1D Particle in a Box 
<br>
Given a quantum mechanical particle in a box, such that the potential outside of the box is infinite, while the potential inside of the box is 0, the analytical solution to the Schrödinger equation is: <br>
<br>
$$ \psi_n(x) = \sqrt{\frac{2}{L}} \sin(\frac{nx\pi}{L})$$ <br>
<br>
The variables in the equation have the following meaning: <br>

$n$ specifies the quantum number of the system from 1 to infinity. <br>
$L$ represents the length of the box in Bohr Radius that the particle is in <br>
<br>
This $\psi_n$ represents the eigen values of the particle in the box. The energies of these eigenvalues are represented by: <br>
$$E_n = \frac{\hbar^2\pi^2n^2}{2mL^2}$$ <br>
<br>
$m$ refers to the mass of the particle in atomic units (a.u.) <br>
<br>
The probability of finding the particle at any particular location in the box is represented by $|\psi|^2$, meaning that the wave function is noramlized such that the probability of having to find the particle at <i>some</i> location within the box must be 1. This concept is matematically expressed as:
$$\int_0^L{|\psi_n|^2 dx} = 1$$ <br>
<br>
The wavefunctions $\psi_n$ are also orthogonal to one another, meaning that a set of $\psi_n$ will be orthonormal to each other.
$$\int_0^L{\psi_i^*(x)\psi_j(x)dx} = \delta_{ij}$$

In [146]:
#import all needed packages here
import math
import ipywidgets as widgets
from ipywidgets import interact, interactive_output, fixed
from scipy.integrate import quad as integrate
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)

In [199]:
#All Functions Defined Here

#Analytical Solution for 1D Particle in a Box
#x is the position of the particle in 1D space in Bohr Radius
#n is the quantum number from 1 to infinity
#L is the box length in Bohr Radius
def psi(x, n, L):
    return math.sqrt( 2 / L ) * math.sin(n * x * math.pi / L)

#------------------------------------------------------------------------

#Function to Compute the Energy of the specified eigenstate
#Note, due to use of atomic units, hbar disappears from the energy 
#equation since it is equal to 1
#n is the specified quantum number
#L is the box length in Bohr Radius
#m is the mass of the particle in box in A.U.
def E(n, L, m):
    return pow(math.pi * n / L, 2) / (2 * m)  

#------------------------------------------------------------------------

#Computes the Energy and Wavefunction data used for graphing
#qn is the list of quantum numbers use in the calculation
#L is the length of the box in Bohr Radius
#m is the mass of the particle in the box in A.U.
#points is the number of points that should be used to perform the computation 
#     The larger the number of points, the higher the resolution of the graph produced
def dataComputation(qn, L, points, m):
    
    #Calculate all computed variables here
    dx = L / points

    yData = [ [] for n in qn]
    prob = [ [] for n in qn ]
    xData = [x * dx for x in range(points)]

    EData = [ [E(n, L, m)] * len(xData) for n in qn]

    #Perform Main Computation Here
    for nIndex, n in enumerate(qn):
        for x in xData: 
            yData[nIndex].append( psi(x, n, L) )
            prob[nIndex].append( abs(psi(x, n, L)) ** 2 )
            
    return xData, yData, EData, prob

#------------------------------------------------------------------------

#Function used to graph the data
#xData is a list of x points that are to be graphed
#yData is a list of y points that are to be graphed
#EData is a list of energy values to graph
#prob is the y values for the probabiliity of finding the particle in any part of the box
#qn is a list of quantum numbers to graph
def graph(xData, yData, EData, prob, qn):
    figure = {
        "data": [], 
        "layout":{
            "xaxis":{"title":"Distance in Bohr Radius"},
            "yaxis":{"title": "Hartrees"},
            "title":{"text":"1D Particle in a Box"}
        }
    }

    figure2 = {
        "data":[],
        "layout":{
            "xaxis":{"title":"Distance in Bohr Radius"},
            "yaxis":{"title":"Psi Squared"},
            "title":{"text":"Wavefunction Probability"}
        }
    }
    
    for nIndex, n in enumerate(qn):
        figure["data"].append(
            {
                "type":"scatter",
                "x":xData,
                "y":[y + EData[nIndex][index] for index, y in enumerate(yData[nIndex])],
                "name": "n=" + str(n) + " Psi",
                "connectgaps": True,
                "mode":"lines"
            }
        )

        figure["data"].append(
           { 
               "type":"scatter",
               "x":xData,
               "y":EData[nIndex],
               "name":"n=" + str(n) + " Energy",
               "connectgaps": True,   
               "mode":"lines",
               "line":{"dash":"dash"}
           }
        )
        
        figure2["data"].append(
            {
                "type":"scatter",
                "x":xData,
                "y":prob[nIndex],
                "mode":"lines",
                "connectgaps":True,
                "name":"n=" + str(n)
            }
        )
    
    iplot(figure)
    iplot(figure2)

#------------------------------------------------------------------------

#main function used to allow the program to run with a graphic interface
#qnInput is the list of quantum numbers directly inputted by the user
#L is the length of the box in Bohr Radius
#points is the number of points to use to compute the data for the graph 
#       The more points used in the computation , the greater the resolution of the graph
#m is the mass of the particle in A.U.
def main(qnInput, L, points, m=1):
    
    qn = []
    
    #perform input checking on qn
    for n in qnInput.split(","):
        
        if(n.isspace() or n == ""):
            continue
        else:
            n.strip()
        
        try:
            qn.append(int(n))
        except:
            print("Warning! Only numbers sparated by commas are considered valid input for quantum" + "numbers!\n" + str(n) + " is not considered a valid quantum number.")
            return
    
    qn = list(set(qn))
    data = [qn, L, points, m, points]
    
    #check that there are data values to compute 
    if(len(qnInput) > 0 and L > 0):
        xData, yData, EData, prob = dataComputation(qn, L, points, m)
        graph(xData, yData, EData, prob, qn)
        main.oldData = data
    
        #Code to display the integral over the squared wavefunction 
        #to demonstrate that the wavefunction is normalized
        mathString = "<font size='5'>"
        for n  in qn:
            psi2 = lambda x : abs(psi(x, n, L)) ** 2
            mathString += "$\int_0^{" + str(L) + "}{|\psi_" + str(n) + "|^2dx} = " + str(
                integrate(psi2, 0, L)[0]) + "$<br>"
        mathString += "</font>"
            
        #display the latex string to the user
        math = widgets.HTMLMath(
            value = mathString,
            
        )
        display(math)

In [200]:
#Quantum Numer Input System
qnUI = widgets.Text(
    value = "1, 2, 3",
    description = "", 
    continous_update = False
)
qnUI.layout.margin = "0 0 0 80px"
qnLabel = widgets.Label("Quantum Number(s)")

#Box Length Input System in Bohr Radius
LUI = widgets.BoundedFloatText(
    value = 0,
    min = 0,
    max = pow(10, 100)
)
LUI.layout.margin = "0 0 0 40px"
LUILabel = widgets.Label("Box Length in Bohr Radius")

#Number of Points Input System, 
#The larger this number the greater the resolution of the graph will be
points = widgets.BoundedIntText(
    min = 0, 
    max = pow(10,100),
    value = 200
)
points.layout.margin = "0 0 0 99px"
pointsLabel = widgets.Label("Number of Points")

#Mass of the Particle in A.U.
m = widgets.BoundedIntText(
    min = 1,
    max = pow(10, 100),
    value = 1,
)
m.layout.margin = "0 0 0 62px"
mLabel = widgets.Label("Mass of Particle in A.U.")

#prepare layout for the input widgets
#then start the widgets to run
layout = widgets.VBox([ widgets.HBox([qnLabel, qnUI]),
                       widgets.HBox([LUILabel, LUI]),
                       widgets.HBox([mLabel, m]),
                       widgets.HBox([pointsLabel, points]),
                      ])
ui = interactive_output(main, {"qnInput":qnUI, "L":LUI, "points":points, "m":m} )
display( 
     layout, ui
 )

Output()